In [1]:
# For model data-feeding
import numpy as np
# For data pre-processing
import pandas as pd
# Pre-ready model: Sequential
from tensorflow.keras.models import Sequential
# Pre-ready for MLP
from tensorflow.keras. layers import Dense
# Pre-ready for Optimiser
from tensorflow.keras. optimizers import RMSprop
# one-hot encoding 용도
from tensorflow.keras.utils import to_categorical

# epoch 가 val_loss를 기준으로 성능개선이 없으면 자동종료
# 가장 좋았던 모델 가중치로 복원하여 과적합을 방지하고 학습시간을 절약하는 용도
from tensorflow.keras.callbacks import EarlyStopping
# Normalising lib
from sklearn.preprocessing import StandardScaler

# 학습할 부분과 테스트 할 부분을 분리
from sklearn.model_selection import train_test_split
# 결과값 정리 용도도
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
!pwd

/content


In [3]:
# 정규화 도구 생성
stdsclr = StandardScaler() # 표준화를 위한 함수를 호출
stdsclr_goal = StandardScaler() # class 값의 표준화를 위한 함수를 호출

# CSV 파일 읽기
raw = pd.read_csv("/content/CarPrice_Assignment.csv")
goal = stdsclr_goal.fit_transform(raw["price"].values.reshape(-1, 1))  # 목표값도 정규화

In [4]:
# 수동 원핫 인코딩 함수
def manual_onehot(_raw, _droplist, tag):
    _droplist.append(tag)
    unique_vals = sorted(_raw[tag].unique())
    val2idx = {val: idx for idx, val in enumerate(unique_vals)}
    mapped_vals = _raw[tag].map(val2idx).values
    onehot = to_categorical(mapped_vals)
    colnames = [f"{tag}_{val}" for val in unique_vals]
    onehot_df = pd.DataFrame(onehot, columns=colnames)
    return onehot_df.astype(float)

In [5]:
#자동 원핫 인코딩 함수
"""
# returns the dummies, register the tag to be replaced
def _get_dummies(_raw, _droplist, tag):
    _droplist.append(tag)
    return pd.get_dummies(_raw[tag], prefix=tag).astype(float)

이후의 코드에서

Encodeds = [                                              # one_hot_encoding 할 변수들을 리스트에 저장 및 함수를 호출하는 코드이다.
            _get_dummies(raw, droplist, "carbody")            # 범주형 변수 'carbody' 를 ont_hor_encoding 하는 과정
            , _get_dummies(raw, droplist, "CarName")          # 범주형 변수 'CarName' 를 ont_hor_encoding 하는 과정
            , _get_dummies(raw, droplist, "drivewheel")       # 범주형 변수 'drivewheel' 를 ont_hor_encoding 하는 과정
            , _get_dummies(raw, droplist, "enginetype")       # 범주형 변수 'enginetype' 를 ont_hor_encoding 하는 과정
            , _get_dummies(raw, droplist, "fuelsystem")       # 범주형 변수 fuelsystem' 를 ont_hor_encoding 하는 과정
            , _get_dummies(raw, droplist, "cylindernumber")   # 범주형 변수 ;cylindernumber' 를 ont_hor_encoding 하는 과정
            ]

이런 형식으로 사용하여 one-hot-encoding
"""

'\n# returns the dummies, register the tag to be replaced\ndef _get_dummies(_raw, _droplist, tag):\n    _droplist.append(tag)\n    return pd.get_dummies(_raw[tag], prefix=tag).astype(float)\n\n이후의 코드에서 \n\nEncodeds = [                                              # one_hot_encoding 할 변수들을 리스트에 저장 및 함수를 호출하는 코드이다.\n            _get_dummies(raw, droplist, "carbody")            # 범주형 변수 \'carbody\' 를 ont_hor_encoding 하는 과정\n            , _get_dummies(raw, droplist, "CarName")          # 범주형 변수 \'CarName\' 를 ont_hor_encoding 하는 과정  \n            , _get_dummies(raw, droplist, "drivewheel")       # 범주형 변수 \'drivewheel\' 를 ont_hor_encoding 하는 과정\n            , _get_dummies(raw, droplist, "enginetype")       # 범주형 변수 \'enginetype\' 를 ont_hor_encoding 하는 과정\n            , _get_dummies(raw, droplist, "fuelsystem")       # 범주형 변수 fuelsystem\' 를 ont_hor_encoding 하는 과정\n            , _get_dummies(raw, droplist, "cylindernumber")   # 범주형 변수 ;cylindernumber\' 를 ont_hor_encoding 하는 과정\n            ]\n

In [6]:
# 전처리 함수 정의
def PreProcess(raw):
    # CarName: Typo fix
    raw["CarName"] = raw["CarName"].str.split(' ').str[0].str.lower()
    # 공백을 기준으로 분리하고, 가장 앞에 있는 단어를 추출 후 소문자로 변환.

    #replace('A', 'B') -> A를 B로 변환/ 5줄 중복이므로 생략
    raw["CarName"] = raw["CarName"].str.replace('volkswagen', 'vw')
    raw["CarName"] = raw["CarName"].str.replace('vokswagen', 'vw')
    raw["CarName"] = raw["CarName"].str.replace('toyouta', 'toyota')
    raw["CarName"] = raw["CarName"].str.replace('maxda', 'mazda')
    raw["CarName"] = raw["CarName"].str.replace('porschece', 'porsche')

    # raw['A'] == 'B' 에서 A가 B의 조건에 따라 True , False 로 변환
    # type 를 float 로 변환하여 이진 코딩을 수행하는 과정 / 3줄 중복이므로 생략
    raw["fueltype"] = (raw["fueltype"] == "gas").astype(float)
    raw["aspiration"] = (raw["aspiration"] == "std").astype(float)
    raw["doornumber"] = (raw["doornumber"] == "two").astype(float)
    raw["symboling"] = raw["symboling"].astype(float)

    # Columns to be replaced
    # Pre-defined was considered "useless"
    droplist = ["car_ID", "enginelocation", "price"]
    # 선형 회귀를 위한 feature 로서, 영향력이 미미해 제거하는 column 변수들
    # car_ID : ID를 붙인것.. 그냥 1~200의 숫자가 나열
    # enginelocation : front 99 rear 1
    # price : goal 이므로 제거거

    Encodeds = [
        # one_hot_encoding 할 변수들을 리스트에 저장 및 함수를 호출
        # manual_onehot(raw, droplist, "A") -> 범주형 변수 'A' 를 ont_hor_encoding 하는 과정
        # 6줄 중복이므로 생략
        manual_onehot(raw, droplist, "carbody"),
        manual_onehot(raw, droplist, "CarName"),
        manual_onehot(raw, droplist, "drivewheel"),
        manual_onehot(raw, droplist, "enginetype"),
        manual_onehot(raw, droplist, "fuelsystem"),
        manual_onehot(raw, droplist, "cylindernumber")
    ]
    # Drop the raw which are processed
    # 데이터 편향이 심한 변수들을 DataFrame 에서 제거
    raw = raw.drop(droplist, axis=1)
    raw_norm = stdsclr.fit_transform(raw)
    # DataFrame 의 모든 열을 평균  = 0, 표준편차 = 1 인 표준정규분포로 표준화하는 코드
    # 그리고 fit 을 통해서 각각의 column 의 평균, 표준 편차를 계산하고, transform 통해서 각각의 평균, 표준편차로 변환

    raw_norm = np.concatenate([raw_norm] + [e.values for e in Encodeds], axis=1)
    # 기존의 DataFrame 에 one_hot_encoding 처리를 한 Encodeds 를 가로로, 즉 열 기준으로 병합

    # 정규화, one_hot_encoding 이 적용된 DataFrame 을 반환
    return raw_norm

In [7]:
# 전처리 실행
raw_norm = PreProcess(raw)
# 함수를 호출하여, 학습에 진행을 위한 표준화,
# one_hot_encoding 가 적용된 raw 를 raw_norm 에 초기화 -> 학습 준비 완료료
rawnp_norm = np.array(raw_norm, dtype=float)
# 학습을 위해서 type 을 DataFrame 에서 numpy_array 로 변경

In [8]:
# 최종 데이터 열
col_len = len(raw_norm[0])
print(f"len: {col_len}")
print(raw_norm[0])

len: 70
[ 1.74347043  0.32879797  0.46929532  1.13038833 -1.6907718  -0.42652147
 -0.84478235 -2.0204173  -0.01456628  0.07444893  0.51907138 -1.83937734
 -0.28834891  0.17448278 -0.26296022 -0.64655303 -0.54605874  1.
  0.          0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.
  1.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  1.          0.          0.          0.          0.          1.
  0.          0.          0.          0.        ]


In [9]:
# 학습용/테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    rawnp_norm, goal, test_size=0.2, random_state=42
    # rawnp_norm -> x, goal-> y 라고 명시해주고
    #test와 train 의 비율을 0.2로 설정
    # random state 는 항상 데이터가 동일하게 섞이도록 시드를 고정한다는 뜻뜻
)

In [10]:
# 모델 정의 (relu + 더 많은 은닉층)

# 비선형 함수/ 활성화 방식은 relu 를 사용한다는 의미
# relu : 음수는 0으로 바꾸고 양수는 그대로 출력
model = Sequential([
    Dense(col_len, activation="relu"), #입력값을 데이터 열 (70개) 만큼의 방식으로 해석한다는 의미
    # Dense(64, activation="relu"), #압축 /고차원
    Dense(32, activation="relu"), #압축 : 핵심적인 패턴만 볼 수 있도록 함
    Dense(1) #가격 예측
])

In [11]:
# EarlyStopping 설정
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
#  val_loss를 기준으로 50번이상 성능개선이 없으면 자동으로 학습을 종료
# 가장 좋았던 모델 가중치로 복원-> 과적합을 방지하고 학습시간을 절약하는 용도

In [12]:
# 모델 컴파일 및 학습
model.compile(loss="mse", optimizer=RMSprop(learning_rate=0.0001))
# 모델 컴파일
# 손실 함수로는 Mean Squares Error 을 사용,
# 최적화는 rmsprop 를 사용.
# 학습율 파라미터를 설정하는 부분

#모델을 실제로 학습시키는 코드
history = model.fit(
    X_train, y_train, # 입력데이터
    epochs=300, #반복 횟수 지정
    batch_size=5, #한 번 학습할 크기 지정
    validation_split=0.2, # 20%를 분리해서 매 epoch 마다 검증손실도 계산
    # -> 일반화가 잘 되는지 확인하고, 조기종료 조건을 알 수 있음
    callbacks=[early_stop], # 학습을 종료할 타이밍이 있다고 알려주기
    verbose=1 # progress bar 와 loss/valloss 를 출력하라는 parameter
)

Epoch 1/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 1.0848 - val_loss: 1.5820
Epoch 2/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.9043 - val_loss: 1.3897
Epoch 3/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.6859 - val_loss: 1.2183
Epoch 4/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.5041 - val_loss: 1.0665
Epoch 5/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.3996 - val_loss: 0.9534
Epoch 6/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.5205 - val_loss: 0.8574
Epoch 7/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.4152 - val_loss: 0.7759
Epoch 8/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2476 - val_loss: 0.7070
Epoch 9/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2933 - val_loss: 0.6461
Epoch 10/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2535 - val_loss: 0.5873
Epoch 11/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.2467 - val_loss: 0.5496
Epoch 12/300
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step

In [13]:
# 테스트 예측 및 정확도 평가
preds = model.predict(X_test)# 학습 이후 모델의 성능을 예측

preds = stdsclr_goal.inverse_transform(preds) #모델이 예측한 결과를 원래 가격 스케일로 돌려놓기

y_test_orig = stdsclr_goal.inverse_transform(y_test.reshape(-1, 1)) #테스트 정답값도 복원

print("✅ learning rate: 0.0001, epochs: 300, batch_size: 5") # 결과화면 캡쳐 (실험 조건 출력력) 를 위해 적음
# 비정규화 RMSE
rmse = np.sqrt(mean_squared_error(y_test_orig, preds))
# 테스트 RMSE (원 단위): 예측값과 실제값의 평균 제곱근 오차 (복원된 값 기준)
print(f"\n✅ 테스트 RMSE (원 단위): {rmse:.2f}") #소숫점 두자리까지 출력

# 정규화된 RMSE도 추가 출력
rmse_norm = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
# 정규화된 값 기준의 RMSE (모델이 내부적으로 사용하는 기준)
print(f"✅ 테스트 RMSE (정규화 단위): {rmse_norm:.4f}") #소숫점 네자리까지 출력

# 비정규화된 loss 추정값 출력 (정규화된 loss 평균 × std^2)
final_loss_norm = history.history['loss'][-1] #최종 학습 손실 값
price_std = stdsclr_goal.scale_[0] #price 표준편차 (원단위로 복원할 때 씀 )
loss_real_unit = final_loss_norm * (price_std ** 2) #최종 loss
print(f"\n✅ 최종 Loss (원 단위 추정): {loss_real_unit:.2f}") # 소숫점 두자리까지 출력

# MAE 출력 (원 단위 & 정규화 단위)
mae = mean_absolute_error(y_test_orig, preds) #절댓값 오차의 평균 (실제 가격 단위)
print(f"\n✅ 테스트 MAE (원 단위): {mae:.2f}") # 소숫점 두자리까지 출력
mae_norm = mean_absolute_error(y_test, model.predict(X_test)) # 정규화된 값 기준의 MAE
print(f"✅ 테스트 MAE (정규화 단위): {mae_norm:.4f}") # 소숫점 네자리까지 출력



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
✅ learning rate: 0.0001, epochs: 300, batch_size: 5

✅ 테스트 RMSE (원 단위): 2582.72
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
✅ 테스트 RMSE (정규화 단위): 0.3241

✅ 최종 Loss (원 단위 추정): 280261.49

✅ 테스트 MAE (원 단위): 1726.42
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
✅ 테스트 MAE (정규화 단위): 0.2166
